In [1]:
from utils1.retreiver_setting import faiss_retriever_loading

account_retriever, business_retriever, business_retriever2, self_retriever = faiss_retriever_loading()

## Self-Query-Retreiver와 일반 Retriever의 답변 정확도 체크

In [8]:
# 질문에 대한 정답 데이터 생성 (답변 문서에 꼭 있어야 하는 키워드)

queries_and_answers = [
    ("삼성바이오로직스 2024년 연결 당기순이익", ["삼성바이오로직스", "2024", "연결", "당기순이익"]),
    ("2024년 LG전자 연구개발 활동", ["2024", "LG전자", "연구개발"]),
    ("2023년 카카오 주요사항 요약", ["2023", "카카오", "주요사항"]),
    ("NAVER 2024년 EPS는 얼마야?", ["NAVER", "2024", "주당순이익"]),
    ("SK하이닉스 2023년 배당금 정보 알려줘", ["SK하이닉스", "2023", "배당금"]),
    ("현대차 2024년 주당액면가액은?", ["현대차", "2024", "주당액면가액"]),
    ("LG화학의 2022년 현금배당 내역", ["LG화학", "2022", "현금배당"]),
    ("카카오 2024년 연구개발 투자 비율은?", ["카카오", "2024", "연구개발", "투자", "비율"]),
    ("삼성전자 2023년 주요사항 정리해줘", ["삼성전자", "2023", "주요사항"]),
    ("NAVER 2024년 연결 실적", ["NAVER", "2024", "연결"]),
    ("LG에너지솔루션의 2023년 EPS", ["LG에너지솔루션", "2023", "주당순이익"]),
    ("SK이노베이션 2024년 주식배당 수익률", ["SK이노베이션", "2024", "주식배당수익률"]),
    ("2024년 현대모비스의 연구개발 성과", ["현대모비스", "2024", "연구개발", "성과"]),
    ("카카오 2022년 별도 재무성과", ["카카오", "2022", "별도"]),
    ("삼성바이오로직스의 2024년 R&D 투자", ["삼성바이오로직스", "2024", "연구개발", "투자"]),
]


In [20]:
from difflib import SequenceMatcher
import re

# 문서에서 단어를 추출하는 함수
def extract_words(text):
    words = re.findall(r'\b\w+\b', text.lower())
    return set(words)

# 수정된 유사도 점수 계산
def evaluate_similarity_score(retriever, queries_and_answers):
    scores = []
    for query, expected_keywords in queries_and_answers:
        docs = retriever.invoke(query)
        all_text = " ".join([doc.page_content for doc in docs])

        # 문서에서 단어 추출
        doc_words = extract_words(all_text)

        # 각 키워드별로 최대 유사도만 계산
        keyword_scores = []
        for kw in expected_keywords:
            # 각 키워드와 가장 유사한 문서 단어 찾기
            max_similarity = 0
            if doc_words:  # 문서에 단어가 있는 경우만
                max_similarity = max(
                    SequenceMatcher(None, kw, word).ratio()
                    for word in doc_words
                )
            keyword_scores.append(max_similarity)

        # 키워드별 평균 유사도
        avg_score = sum(keyword_scores) / len(keyword_scores) if keyword_scores else 0
        scores.append(avg_score)

    return sum(scores) / len(scores) if scores else 0

# 키워드 포함 점수
def evaluate_contains_score(retriever, queries_and_answers):
    scores = []
    for query, expected_keywords in queries_and_answers:
        docs = retriever.invoke(query)
        all_text = " ".join([doc.page_content for doc in docs])

        # 문서에서 단어 추출
        doc_words = extract_words(all_text)

        # 키워드가 문서에서 포함된 횟수 세기
        hit_count = sum(1 for kw in expected_keywords if kw in doc_words)

        # 키워드 포함 비율
        score = hit_count / len(expected_keywords) if expected_keywords else 0
        scores.append(score)  # 각 질문에 대해 점수 계산

    return sum(scores) / len(scores) if scores else 0  # 전체 평균

In [21]:
# 점수 체크

score1_sim = evaluate_similarity_score(business_retriever2, queries_and_answers)
score1_contain = evaluate_contains_score(business_retriever2, queries_and_answers)

score2_sim = evaluate_similarity_score(self_retriever, queries_and_answers)
score2_contain = evaluate_contains_score(self_retriever, queries_and_answers)


print(f"📊 [No Metadata 필터]")
print(f" - 유사도 평가 점수: {score1_sim:.3f}")
print(f" - 키워드 포함 점수: {score1_contain:.3f}")

print(f"📊 [With Self-Query Metadata 필터]")
print(f" - 유사도 평가 점수: {score2_sim:.3f}")
print(f" - 키워드 포함 점수: {score2_contain:.3f}")


📊 [No Metadata 필터]
 - 유사도 평가 점수: 0.764
 - 키워드 포함 점수: 0.323
📊 [With Self-Query Metadata 필터]
 - 유사도 평가 점수: 0.777
 - 키워드 포함 점수: 0.323


In [32]:
queries_and_answers = [
    ("삼성전자의 2024년 연결 재무제표에서 매출액과 영업이익을 포함한 주요 재무 성과는 어떻게 되나요?", ["삼성전자", "2024","매출액", "영업이익"]),
    ("2023년 SK하이닉스의 배당금 지급 내역과 실적에 대한 내용을 알려주세요.", ["SK하이닉스", "2023", "배당금", "실적"]),
    ("2024년 현대차의 순이익과 함께 주요 자산 항목은 어떻게 변화했나요?", ["현대차", "2024", "순이익", "자산"]),
    ("LG에너지솔루션 2023년 주요사항 중 중요한 결산 내용과 관련된 정보를 제공해주세요.", ["LG에너지솔루션", "2023", "주요사항", "결산"]),
    ("2024년 KB금융의 연간 배당금 지급 내역을 포함하여 배당정책에 대해 알려주세요.", ["KB금융", "2024", "배당정책"])]


In [33]:
# 점수 체크

score1_sim = evaluate_similarity_score(business_retriever2, queries_and_answers)
score1_contain = evaluate_contains_score(business_retriever2, queries_and_answers)

score2_sim = evaluate_similarity_score(self_retriever, queries_and_answers)
score2_contain = evaluate_contains_score(self_retriever, queries_and_answers)


print(f"📊 [No Metadata 필터]")
print(f" - 유사도 평가 점수: {score1_sim:.3f}")
print(f" - 키워드 포함 점수: {score1_contain:.3f}")

print(f"📊 [With Self-Query Metadata 필터]")
print(f" - 유사도 평가 점수: {score2_sim:.3f}")
print(f" - 키워드 포함 점수: {score2_contain:.3f}")


📊 [No Metadata 필터]
 - 유사도 평가 점수: 0.734
 - 키워드 포함 점수: 0.200
📊 [With Self-Query Metadata 필터]
 - 유사도 평가 점수: 0.636
 - 키워드 포함 점수: 0.100
